In [1]:
import numpy as np
import pandas as pd

In [14]:
df = pd.read_csv("./diabetes.csv")

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [15]:
df.corr()["Outcome"].sort_values()

,Outcome
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
DiabetesPedigreeFunction,0.173844
Pregnancies,0.221898
Age,0.238356
BMI,0.292695
Glucose,0.466581
Outcome,1.000000


In [16]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [18]:
X = scaler.fit_transform(X)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [53]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout

In [22]:
model = Sequential(
    [Input(shape=(8,)), Dense(32, activation="relu"), Dense(1, activation="sigmoid")]
)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [23]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.6790 - loss: 0.6662 - val_accuracy: 0.6818 - val_loss: 0.6474
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6822 - loss: 0.6323 - val_accuracy: 0.7273 - val_loss: 0.5997
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7215 - loss: 0.5838 - val_accuracy: 0.7597 - val_loss: 0.5658
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7307 - loss: 0.5573 - val_accuracy: 0.7727 - val_loss: 0.5417
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7476 - loss: 0.5377 - val_accuracy: 0.7792 - val_loss: 0.5241
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7463 - loss: 0.5250 - val_accuracy: 0.7922 - val_loss: 0.5111
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7495 - loss: 0.5208 - val_accuracy: 0.7922 - val_loss: 0.5010
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7533 - loss: 0.4963 - val_accuracy: 0.7922 - 

In [27]:
import kerastuner as kt

In [28]:
from keras.src.optimizers import optimizer


def build_model(hp):

    model = Sequential()
    model.add(Dense(32, activation="relu", input_shape=(8,)))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = hp.Choice("optimzer", values=["adam", "sgd", "rmsprop", "adadelta"])
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [29]:
tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=5)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 14s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'optimzer': 'rmsprop'}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.fit(
    X_train,
    y_train,
    batch_size=32,
    initial_epoch=6,
    epochs=100,
    validation_data=(X_test, y_test),
)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.7685 - loss: 0.5033 - val_accuracy: 0.7922 - val_loss: 0.4958
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7577 - loss: 0.4869 - val_accuracy: 0.7922 - val_loss: 0.4857
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7863 - loss: 0.4721 - val_accuracy: 0.7922 - val_loss: 0.4794
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7703 - loss: 0.4718 - val_accuracy: 0.7922 - val_loss: 0.4752
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7905 - loss: 0.4692 - val_accuracy: 0.7922 - val_loss: 0.4712
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7904 - loss: 0.4469 - val_accuracy: 0.7922 - val_loss: 0.4676
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7660 - loss: 0.4712 - val_accuracy: 0.7922 - val_loss: 0.4667
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7836 - loss: 0.4451 - val_accuracy: 0.79

In [35]:
def build_model(hp):
    model = Sequential()

    units = hp.Int("units", min_value=8, max_value=128, step=8)
    model.add(Dense(units, activation="relu", input_shape=(8,)))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [36]:
tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, directory="some_directory"
)

In [37]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 18s


In [38]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [39]:
model = tuner.get_best_models(num_models=1)[0]

In [40]:
model.fit(
    X_train,
    y_train,
    batch_size=32,
    initial_epoch=6,
    epochs=100,
    validation_data=(X_test, y_test),
)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.7708 - loss: 0.5238 - val_accuracy: 0.7922 - val_loss: 0.4948
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7845 - loss: 0.5002 - val_accuracy: 0.7857 - val_loss: 0.4818
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7473 - loss: 0.5117 - val_accuracy: 0.7857 - val_loss: 0.4759
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7979 - loss: 0.4494 - val_accuracy: 0.7857 - val_loss: 0.4723
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7541 - loss: 0.4957 - val_accuracy: 0.7792 - val_loss: 0.4700
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7540 - loss: 0.4845 - val_accuracy: 0.8052 - val_loss: 0.4683
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7724 - loss: 0.4738 - val_accuracy: 0.8052 - val_loss: 0.4660
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7891 - loss: 0.4451 - val_accuracy: 0.80

In [41]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72, activation="relu", input_shape=(8,)))

    for i in range(hp.Int("num_layers", min_value=1, max_value=10)):
        model.add(Dense(72, activation="relu"))

    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [42]:
tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, directory="mydir"
)

In [43]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 26s


In [44]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [45]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [46]:
model.fit(
    X_train, y_train, epochs=100, initial_epoch=5, validation_data=(X_test, y_test)
)

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.7707 - loss: 0.4768 - val_accuracy: 0.8052 - val_loss: 0.4616
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7784 - loss: 0.4586 - val_accuracy: 0.7792 - val_loss: 0.4749
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7888 - loss: 0.4595 - val_accuracy: 0.7727 - val_loss: 0.4755
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7899 - loss: 0.4292 - val_accuracy: 0.7727 - val_loss: 0.4693
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7952 - loss: 0.4368 - val_accuracy: 0.8052 - val_loss: 0.4669
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7647 - loss: 0.4303 - val_accuracy: 0.7792 - val_loss: 0.4945
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7944 - loss: 0.4349 - val_accuracy: 0.7857 - val_loss: 0.4670
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7885 - loss: 0.4139 - val_accuracy: 0.798

In [58]:
def build_model(hp):
    model = Sequential()

    counter = 0

    for i in range(hp.Int("num_layers", min_value=1, max_value=10)):
        if counter == 0:
            model.add(
                Dense(
                    hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice(
                        "activation" + str(i), values=["relu", "tanh", "sigmoid"]
                    ),
                    input_shape=(8,),
                ),
            )
            model.add(
                Dropout(
                    hp.Choice(
                        "dropout_rate" + str(i),
                        values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                    )
                )
            )
        else:
            model.add(
                Dense(
                    hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice(
                        "activation" + str(i), values=["relu", "tanh", "sigmoid"]
                    ),
                )
            )
            model.add(
                Dropout(
                    hp.Choice(
                        "dropout_rate" + str(i),
                        values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                    )
                )
            )
        counter += 1

    model.add(Dense(1, activation="sigmoid"))
    model.compile(
        optimizer=hp.Choice(
            "optimizer", values=["rmsprop", "adam", "sgd", "nadam", "adadelta"]
        ),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [62]:
tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, directory="randomdi3"
)

In [63]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 11s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 01m 08s


In [64]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 88,
 'activation0': 'tanh',
 'dropout_rate0': 0.4,
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'dropout_rate1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout_rate2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout_rate3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout_rate4': 0.1}

In [65]:
model = tuner.get_best_models(num_models=1)[0]

In [66]:
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    initial_epoch=6,
    validation_data=(X_test, y_test),
)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 165ms/step - accuracy: 0.6574 - loss: 0.6791 - val_accuracy: 0.6429 - val_loss: 0.6773
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6173 - loss: 0.6811 - val_accuracy: 0.6429 - val_loss: 0.6744
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6336 - loss: 0.6764 - val_accuracy: 0.6429 - val_loss: 0.6718
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6508 - loss: 0.6704 - val_accuracy: 0.6429 - val_loss: 0.6694
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6585 - loss: 0.6669 - val_accuracy: 0.6429 - val_loss: 0.6672
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6389 - loss: 0.6683 - val_accuracy: 0.6429 - val_loss: 0.6654
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6718 - loss: 0.6602 - val_accuracy: 0.6429 - val_loss: 0.6635
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6641 - loss: 0.6569 - val_accuracy: 0.6